In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sys
print(sys.path)

['', '/Users/shandou/anaconda/envs/mlnd/lib/python36.zip', '/Users/shandou/anaconda/envs/mlnd/lib/python3.6', '/Users/shandou/anaconda/envs/mlnd/lib/python3.6/lib-dynload', '/Users/shandou/anaconda/envs/mlnd/lib/python3.6/site-packages', '/Users/shandou/anaconda/envs/mlnd/lib/python3.6/site-packages/cycler-0.10.0-py3.6.egg', '/Users/shandou/anaconda/envs/mlnd/lib/python3.6/site-packages/IPython/extensions', '/Users/shandou/.ipython']


In [3]:
import sys
import os
import random
import gc
import subprocess
from pprint import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
plt.style.use('./stylelib/custom.mplstyle')

%reload_ext autoreload
%autoreload 2
import preprocessing
import plotter

---

## 1 Data inspection and loading
### 1.1 Downsample training data
Only 0.5% of the all training records is used for exploratory data analysis

In [4]:
%%time

dir_data = './data'
from_scratch = False

csv_train_raw = os.path.join(dir_data, 'train.csv')
csv_train = os.path.join(dir_data, 'train_sample.csv')
if from_scratch:
    nlines_raw, nlines_reduced = preprocessing.csv_randomized_downsamp(
        csv_in=csv_train_raw, csv_out=csv_train, fraction=0.005
    )

CPU times: user 16 µs, sys: 0 ns, total: 16 µs
Wall time: 21 µs


### 1.2 Load data into dataframe
#### (1) Field inspections
For efficiency concerns, we use shell commands instead of pandas operations.

In [5]:
# Quick check of training data fields by calling system shell command
!head -2 ./data/train_sample.csv

ip,app,device,os,channel,click_time,attributed_time,is_attributed
93766,3,1,19,205,2017-11-07 10:28:10,,0


In [6]:
# Quick check of test data fields by calling system shell command
!head -2 ./data/test.csv

click_id,ip,app,device,os,channel,click_time
0,5744,9,1,3,107,2017-11-10 04:00:00


**[CAUTION]** We should be very careful about the use of datetime feature `click_time`. The simple check above indicates the date value of the training and testing data don't overlap. For this reason, we should NOT use raw date or day-of-the-week as a feature when training our model.

**[Notes] Field selections** <br>
Note that from above that training and testing data don't share the same fields. <br>
To prepare data for subsequent processing, we only preserve fields that are common in both training and testing data. One exception is the `is_attributed` field in training data; It is the prediction target and thus needs to be preserved.

In [7]:
# Extract field names from training and testing data
fields_train = preprocessing.csv_list_fields(csv_in=csv_train)
csv_test = os.path.join(dir_data, 'test.csv')
fields_test = preprocessing.csv_list_fields(csv_in=csv_test)

# Extract fields shared by both training and testing data
fields_use = list(set(fields_train) & (set(fields_test)))
print('Data fields shared by both training and testing data:\n', fields_use)

Data fields shared by both training and testing data:
 ['channel', 'os', 'click_time', 'app', 'device', 'ip']


#### (2) Load into dataframe

In [8]:
# Load randomly sampled data subset into pandas dataframe,
# and sort by click time (and reset index)
df = pd.read_csv(
    csv_train, usecols=fields_use + ['is_attributed'],
    parse_dates=['click_time']
).sort_values(by='click_time').reset_index(drop=True)

# Convert click time from UTC to local time
df['click_time'] = (
    pd.DatetimeIndex(df['click_time']).tz_localize('utc')
    .tz_convert('Asia/Shanghai')
)

In [9]:
# Inspect first few lines
df.head(3)

,ip,app,device,os,channel,click_time,is_attributed
0,143257,3,1,15,379,2017-11-06 23:41:21+08:00,0
1,186555,3,1,13,379,2017-11-06 23:54:57+08:00,0
2,119289,3,1,13,379,2017-11-06 23:57:20+08:00,0


---
## 2. Exploratory data analysis
### 2.1 Do we have imbalanced classes?

In [10]:
# Inspect data size
nclick_total =len(df)
percentage_pos = (df['is_attributed'].sum()) / nclick_total * 100
print('Percentage of positive target = {:.3f}%'.format(percentage_pos))

Percentage of positive target = 0.248%


**[CAUTION] Class imbalance**<br>
Given that only 0.25% of the records has positive target values (`df['is_attributed'] == 1`), we have **an extreme case of class imbalance** at hand.

### 2.2 Convert datetime variable to usable form
To convert datetime to a more usable form, we extract hour-of-the-day information

In [11]:
# Extract hour of the day
df['click_hour'] = df['click_time'].dt.hour

# Drop raw `click_time`
df.drop(columns=['click_time'], inplace=True)

In [12]:
df.head()

,ip,app,device,os,channel,is_attributed,click_hour
0,143257,3,1,15,379,0,23
1,186555,3,1,13,379,0,23
2,119289,3,1,13,379,0,23
3,62129,12,1,13,245,0,0
4,2895,9,2,58,244,0,0


### 2.3 Train-test split, and set aside testing data

In [ ]:
# Split the original dataframe into in-sample training and testing sets
# Because stratified sampling is the default option of sklearn, it is not
# explicitly set
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

# Double check class ratios after train-test split
print('Training data (pos%):', 100 * df_train['is_attributed'].sum() / len(df_train))
print('Testing data (pos%):', 100 * df_test['is_attributed'].sum() / len(df_test))

### 2.4 Inspect variable distributions of training data

In [ ]:
# Count the amount of unique values
df_counts = pd.DataFrame()
df_counts['n_unique'] = df_train.nunique()
df_counts['n_unique (%)'] = 100 * df_counts['n_unique'] / len(df_train)
df_counts.T

**[TAKE AWAY]**<br>
* Without exception, the total number of unique values for each of these variables is markedly smaller than the total number of clicks. This indicates that many-to-one mapping is typical between clicks and these variables. This makes sense because a single user can generate multiple clicks.
* The amount of distinct values in categorical features is very high. We will need to make major efforts in feature engineering.

### 2.5 Apply target-guided encoding to categorical features

In [ ]:
features = list(df_train.columns)
features.remove('is_attributed')
pd.options.mode.chained_assignment = None

for ft in features:
    mapper = preprocessing.mapper_category2woe(df_train, ft)
    df_train['{}_prob'.format(ft)] = df_train[ft].map(mapper)
    df_test['{}_prob'.format(ft)] = df_test[ft].map(mapper)

### 2.6 Check dataframes after encoding

Note that mapping used for encoding is generated with training data and then propagated to testing data. 
Because there are non-overlap variables between training and testing data, and encoded testing data are going to have missing values. Let's have a look:

In [ ]:
# Check presence of missing value after encoding
df_nulls = pd.DataFrame()
df_nulls['nan_train(%)'] = 100 * (df_train.isnull().sum() / len(df_train))
df_nulls['nan_test(%)'] = 100 * (df_test.isnull().sum() / len(df_test))
df_nulls.T

As expected, for `*_woe` variables, the testing data have missing values that requires clean up. We will test a number of approaches later, and use simply dropping the value as a baseline for comparisons.

### 2.7 Check Pearson's correlation coefficient

In [ ]:
# Exclude target variable from correlation calculation
cols_corr = [x for x in df_train.columns if 'prob' in x]

# Plot positive and negative cases in separate heatmaps
fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(8, 4))
fig.tight_layout()
for i, axi in enumerate(ax):
    corr_matrix = df_train[df_train['is_attributed'] == i][cols_corr].corr()
    mask = np.zeros_like(corr_matrix, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    sns.heatmap(
        corr_matrix, mask=mask, cmap='RdBu', vmin=-1, vmax=1,
        square=True, linewidths=.5, annot=True, fmt='.2f',ax=axi, cbar=False
    )
    axi.set(title='is_attributed = {}'.format(i))
plt.show();

---
## 3. Machine learning

In [ ]:
cols = [x for x in df_train.columns if 'prob' in x and 'ip_' not in x]
df_train.dropna(axis=0, inplace=True)
print(cols)
X_train, y_train = (df_train[cols], df_train['is_attributed'])
df_test.dropna(axis=0, inplace=True)
X_test, y_test = (df_test[cols], df_test['is_attributed'])

In [ ]:
print(X_train.shape, X_test.shape)

In [ ]:
X_train.describe()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import (StandardScaler, MinMaxScaler)
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import (cross_val_score, RandomizedSearchCV)

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train.values, y_train.values)

In [ ]:
y_pred2 = model.predict(X_train)
auc = roc_auc_score(y_train, y_pred2)
print(auc)

In [ ]:
y_pred = model.predict(X_test)
auc = roc_auc_score(y_test, y_pred)
print(auc)

In [ ]:
model = RandomForestClassifier(class_weight='balanced', n_estimators=50, max_depth=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
auc = roc_auc_score(y_test, y_pred)
print(auc)

In [ ]:
import xgboost as xgb
train_matrix = xgb.DMatrix(data=X_train, label=y_train)
test_matrix = xgb.DMatrix(data=X_test, label=y_test)
params = dict(objective='binary:logistic', max_depth=8)



model = xgb.XGBClassifier(params=params, seed=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
auc = roc_auc_score(y_test, y_pred)
print(auc)

In [ ]:
from sklearn.feature_selection import RFE
estimator = DecisionTreeClassifier()
selector = RFE(estimator, step=1)
selector = selector.fit(X_train, y_train)

In [ ]:
print(selector.support_)
print(selector.ranking_)

In [ ]:
col_array = np.array(cols)
col_array

In [ ]:
cols = col_array[selector.support_]

In [ ]:
X_train, y_train = (df_train[cols], df_train['is_attributed'])
X_test, y_test = (df_test[cols], df_test['is_attributed'])

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
auc = roc_auc_score(y_test, y_pred)
print(auc)

X_train.shape